In [2]:
import torch
import torch.optim as optim
import torch.nn as nn

import dataset_maker as ds
import data_loader as loaders_handler
import network_model_service as nn_service
import plot_manager as pl

import network_classes as networks
import parameters as constant

ModuleNotFoundError: No module named 'dataset_maker'

In [4]:
nn_model = networks.SimpleNetwork()
nn_model.load_state_dict(torch.load("dH_model"))

<All keys matched successfully>

In [5]:
butch_size = 64
epoch_number = 400
dataset = ds.get_dataset_from_excel_file("ML_Stepan.xlsx", label_column_number=constant.dH_column_number,
                                             begin_feature_column=constant.begin_feature_column,
                                             end_feature_column=constant.end_feature_column,
                                             dna_to_numeric_strategy=None, first_row=constant.first_row)
train_loader, val_loader = loaders_handler.get_train_and_val_loaders(dataset, butch_size)
butch, ground_truth_labels = next(iter(val_loader))


In [7]:
prediction = nn_model(butch)
torch.mean(torch.abs(prediction - ground_truth_labels))
torch.concatenate((prediction, ground_truth_labels), axis=1)

tensor([[-113.1516, -115.2000],
        [ -50.6165,  -59.6000],
        [ -94.5640,  -89.9000],
        [ -57.1960,  -57.7000],
        [ -56.9722,  -60.7000],
        [ -61.2815,  -63.5000],
        [ -63.3834,  -60.0000],
        [-112.1501, -104.4000],
        [ -50.3690,  -49.2000],
        [ -62.4384,  -61.6000],
        [ -64.2676,  -60.4000],
        [ -60.9432,  -60.6000],
        [ -55.1495,  -50.2000],
        [ -61.9482,  -68.8000],
        [-117.7092, -104.4000],
        [ -57.9408,  -57.0000],
        [ -62.4384,  -61.6000],
        [ -61.5580,  -62.6000],
        [ -63.9127,  -66.2000],
        [ -60.9432,  -67.9000],
        [ -89.1698,  -93.7000],
        [ -64.2676,  -71.0000],
        [ -64.1019,  -67.5000],
        [ -66.5025,  -64.1000],
        [ -59.0515,  -57.6000],
        [ -47.7542,  -38.6000],
        [ -49.1345,  -50.7000],
        [ -52.7683,  -51.7000],
        [ -61.9129,  -58.5000],
        [ -71.2245,  -67.4000],
        [ -58.5914,  -56.8000],
        

In [ ]:

import numpy as np
import torch.optim as optim
import torch.nn as nn
import data_loader as loader
import network_service as nn_service
import prediction_analyzer as pl

import network_classes as networks
import excel_parameters as constant
import pandas as pd
import loss
import butch_handlers
from torch.optim.lr_scheduler import StepLR, ReduceLROnPlateau

# butch_size = 64
    butch_size = 64
    dG_coefficients_multiplier = loader.make_coefficients_multiplier(loader.dG_analytic_coefficients)
    dataset = loader.get_dataset_from_excel_file("ML_Stepan.xlsx", label_column_number=constant.dG_column_number,
                                                 begin_feature_column=constant.begin_feature_column,
                                                 end_feature_column=constant.end_feature_column,
                                                 is_dna_analysis=False,
                                                 data_handler=dG_coefficients_multiplier,
                                                 first_row=constant.first_row)
    #dna data
    # dataset = loader.get_dataset_from_excel_file("ML_Stepan.xlsx", label_column_number=constant.dH_column_number,
    #                                          begin_feature_column=constant.dna_feature_column_number,
    #                                          end_feature_column=None,
    #                                          dna_to_numeric_strategy=loader.make_2d_data_from_text_dna,
    #                                          first_row=constant.first_row)
    # train service
    train_loader, val_loader = loader.get_train_and_val_loaders(dataset, butch_size)
    epoch_number = 5000
    nn_model = networks.linear_classifier()
    # loss_function = loss.complementary_normalized_l1_loss(complement_coefficient=0.1)
    loss_function = loss.normalized_l1_loss
    # butch_handler = butch_handlers.complementarity_butch_manager(
    #     butch_size=butch_size, complementarity_additive_size=16,
    #     complementary_pair_maker=butch_handlers.complementary_pair_maker_from_2d)
    butch_handler = butch_handlers.butch_noise_adder(mean=0, sigma=0.0, noise_ground_truth=True, noise_butch=False)
    optimizer = optim.Adam(nn_model.parameters(), lr=0.001)
    history_manager = nn_service.train_history_manager(epoch_number)
    epoch_painter = nn_service.epoch_painter(drawing_function=print, epoch_number_per_drawing=10)
    # scheduler = StepLR(optimizer, step_size=5, gamma=0.1)  # reduce lr every n epochs
    scheduler = ReduceLROnPlateau(optimizer, 'min')  # reduce lr on a plateu
    is_convolution_training = False
    train_manager = nn_service.network_train_service(nn_model, loss_function, optimizer, scheduler,
                                                     butch_handler=butch_handler,
                                                     is_convolution_training=is_convolution_training,
                                                     is_relative_error=True)
    # training
    nn_service.train_model(train_manager, train_loader, val_loader, epoch_number, epoch_painter, history_manager)
    # print(np.min(val_error_history[val_error_history > 0]))
    print(history_manager.get_min_val_error())
    nn_service.plot_train_history(*history_manager.get_train_history())

    # testing
    test_features, test_labels = dataset.tensors
    if is_convolution_training:
        prediction = nn_model(test_features[:, None, :])
    else:
        prediction = nn_model(test_features)
    absolute_error = nn_service.get_prediction_error(prediction, test_labels, is_relative_error=False,
                                                     is_absolute_value=False)
    relative_error = nn_service.get_prediction_error(prediction, test_labels, is_relative_error=True,
                                                     is_absolute_value=False)
    pl.draw_error_histograms("some network", pl.Graph("absolute error", absolute_error), pl.Graph("relative error", relative_error))
    pl.draw_correlation("some network", pl.CorrelationPair(prediction, test_labels, "test"))

    # class train_features:
    #     def __init__(self, network_name, loss_name):
    #         self.network_name = network_name
    #         self.loss_name = loss_name
    #
    #     def __hash__(self):
    #         return hash((self.network_name, self.loss_name))
    #
    #     def __eq__(self, other):
    #         return (self.network_name, self.loss_name) == (self.network_name, self.loss_name)
    #
    #
    # def prediction_draw(prediction, test_labels, graph_name):
    #     absolute_error = nn_service.get_prediction_error(prediction, test_labels, is_relative_error=False,
    #                                                      is_absolute_value=False)
    #     relative_error = nn_service.get_prediction_error(prediction, test_labels, is_relative_error=True,
    #                                                      is_absolute_value=False)
    #     pl.draw_error_histograms(graph_name, pl.Graph("absolute error", absolute_error), pl.Graph("relative error", relative_error))
    #     pl.draw_correlation(graph_name, pl.CorrelationPair(prediction, test_labels, "test"))
    #
    #
    # # nearest neighbour testing
    # dataset = loader.get_dataset_from_excel_file("ML_Stepan.xlsx", label_column_number=constant.dG_column_number,
    #                                              begin_feature_column=constant.dna_feature_column_number,
    #                                              end_feature_column=None,
    #                                              is_dna_analysis=True,
    #                                              data_handler=loader.make_2d_data_from_text_dna,
    #                                              first_row=constant.first_row)
    # butch_size = 64
    # test_features, test_labels = dataset.tensors
    # train_loader, val_loader = loader.get_train_and_val_loaders(dataset, butch_size)
    # epoch_number = 5000
    # networks = {"dna2d_lc": networks.dna_2d_linear_classifier(),
    #             "dna2d_two_layers": networks.dna_2d_two_layer_network(), "dna2d_conv net": networks.ConvNetwork2d(), }
    #
    # losses = {"normalized_l1_loss": loss.normalized_l1_loss, "normalized_l2_loss": loss.normalized_l2_loss}
    # networks_error = {}
    # for current_network_name, current_network in networks.items():
    #     for current_loss_name, current_loss in losses.items():
    #         network_features = train_features(current_network_name, current_loss_name)
    #         print(current_network_name, current_loss_name)
    #         optimizer = optim.Adam(current_network.parameters(), lr=0.001)
    #         history_manager = nn_service.train_history_manager(epoch_number)
    #         scheduler = ReduceLROnPlateau(optimizer, 'min')
    #         is_convolution_training = False
    #         if current_network_name == "dna2d_conv net":
    #             is_convolution_training = True
    #         train_manager = nn_service.network_train_service(current_network, current_loss, optimizer, scheduler,
    #                                                          butch_handler=None,
    #                                                          is_convolution_training=is_convolution_training,
    #                                                          is_relative_error=True)
    #         nn_service.train_model(train_manager, train_loader, val_loader, epoch_number, epoch_painter=None,
    #                                history_manager=history_manager)
    #         print("network trained")
    #         train_error, val_error = history_manager.train_val_min_error()
    #         networks_error[network_features] = train_error, val_error
    #         if is_convolution_training:
    #             prediction = current_network(test_features[:, None, :])
    #         else:
    #             prediction = current_network(test_features)
    #         prediction_draw(prediction, test_labels, current_network_name + " " + current_loss_name)
    #
    # for network_features, error_value in networks_error.items():
    #     print(network_features.network_name, network_features.loss_name, " ", error_value)
